In [1]:
%load_ext autoreload
%autoreload 2

import os
from jax import config
config.update("jax_enable_x64", True)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

import tensorflow_datasets as tfds  # TFDS for MNIST
import tensorflow as tf             # TensorFlow operations

# from image_distribution_models import PixelCNN

from cleanplots import *
import jax.numpy as np
from jax.scipy.special import logsumexp
import numpy as onp





2023-10-21 13:23:43.417111: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-21 13:23:43.930267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/nvidia/cublas/lib:/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/nvidia/cuda_cupti/lib:/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/nvidia/cuda_nvcc/lib:/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/nvidia/cuda_nvrtc/lib:/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/python3.10/site-packages/nvidia/cuda_runtime/lib:/home/hpinkard_waller/mambaforge/envs/phenotypes/lib/pyth

In [3]:
#MNIST

# (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()


# import numpy as onp
# train_images = onp.random.randint(0, 255, train_images.shape).astype('uint8')

# # make some bimodal IID data
# train_images = onp.where(onp.random.rand(*train_images.shape) > 0.5, 250, 5) + onp.random.normal(0, 10, train_images.shape)



In [2]:
from encoding_information.bsccm_utils import *
from encoding_information.image_utils import *

bsccm = BSCCM('/home/hpinkard_waller/data/BSCCM/')

# load images, extract patches, and compute cov mats
edge_crop = 32
patch_size = 10
num_images = 20000
num_images = 5000

channel = 'LED119'
eigenvalue_floor = 1e0

num_test_images = 500

all_images = load_bsccm_images(bsccm, channel=channel, num_images=num_images + num_test_images, edge_crop=edge_crop, median_filter=False)
images = all_images[:num_images]
test_images = all_images[num_images:]
test_patches = extract_patches(test_images, patch_size, num_patches=num_test_images, seed=0)

num_patches = 5000
seed = 10

patches = extract_patches(images, patch_size, num_patches=num_patches, seed=seed)


Opening BSCCM
Opened BSCCM


In [21]:
from encoding_information.models.gaussian_process import StationaryGaussianProcess

stationary_gp = StationaryGaussianProcess(patches, eigenvalue_floor=1e-3)
val_loss_history = stationary_gp.fit(patches, max_epochs=5, eigenvalue_floor=1e-3)          

Computing loss: 8it [00:00, 10.67it/s]


Initial validation NLL: 6148.97


Epoch 1: 100%|██████████| 1/1 [00:04<00:00,  4.57s/it]
Computing loss: 8it [00:00, 99.44it/s]


Epoch 1: validation NLL: 4.59


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 43.19it/s]
Computing loss: 8it [00:00, 99.25it/s]


...

Epoch 3: validation NLL: 4.54


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 47.97it/s]
Computing loss: 8it [00:00, 108.08it/s]


Epoch 4: validation NLL: 4.54


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 46.11it/s]
Computing loss: 8it [00:00, 106.86it/s]


Epoch 5: validation NLL: 4.54


In [3]:
from encoding_information.models.pixel_cnn import PixelCNN

num_hidden_channels = 64
num_mixture_components = 40

pixel_cnn = PixelCNN(num_hidden_channels=num_hidden_channels, num_mixture_components=num_mixture_components)
pixel_cnn.fit(patches, max_epochs=25)          


2023-10-21 13:24:45.654261: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2023-10-21 13:24:45.654288: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:811] Conv: (f64[10,128,10,1]{3,2,1,0}, u8[0]{0}) custom-call(f64[10,64,10,1]{3,2,1,0}, f64[128,64,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}
2023-10-21 13:24:50.664051: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Computing loss: 0it [00:00, ?it/s]2023-10-21 13:24:51.213805: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback alg

Initial validation NLL: 124.32


Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]2023-10-21 13:24:57.185080: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2023-10-21 13:24:57.185105: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:811] Conv: (f64[64,128,1,3]{3,2,1,0}, u8[0]{0}) custom-call(f64[64,64,10,10]{3,2,1,0}, f64[128,64,10,10]{3,2,1,0}), window={size=10x10 stride=2x2 pad=0_0x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}
2023-10-21 13:24:57.246145: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:808] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2023-10-21 13:24:57.246170: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:811] Conv: (f64[64,128,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f64[64,64,10,10]{3,2,1,0}, f64[128

Epoch 1: validation NLL: 5.57


Epoch 2: 100%|██████████| 100/100 [00:08<00:00, 12.48it/s]
Computing loss: 15it [00:00, 38.49it/s]


...

Epoch 21: 100%|██████████| 100/100 [00:07<00:00, 13.02it/s]
Computing loss: 15it [00:00, 39.51it/s]


Epoch 21: validation NLL: 4.60


Epoch 22: 100%|██████████| 100/100 [00:07<00:00, 13.04it/s]
Computing loss: 15it [00:00, 40.01it/s]

Epoch 22: validation NLL: 4.60


[124.31939598576801,
 5.574232822757545,
 5.106379670450322,
 4.7445735053804405,
 4.622940856026082,
 4.583594469338777,
 4.5625859331098395,
 4.578762943963583,
 4.55931019707213,
 4.606637961169037,
 4.521399345447587,
 4.532886622648063,
 4.514651594317875,
 4.527822992543543,
 4.531505795710428,
 4.530609383199943,
 4.5280100821944576,
 4.52451931965293,
 4.526286731362355,
 4.558799902405855,
 4.541464918750059,
 4.598804847170132,
 4.599318598845315]

## Compare test set likelihood and samples from both models

In [31]:
stationary_gp_nll = stationary_gp.compute_negative_log_likelihood(test_patches)
pixel_cnn_nll = pixel_cnn.compute_negative_log_likelihood(test_patches)
print(f"Stationary GP NLL: {stationary_gp_nll}")
print(f"PixelCNN NLL: {pixel_cnn_nll}")

precomputing masks and variances: 100%|██████████| 10/10 [00:00<00:00, 178.22it/s]


evaluating likelihood


computing log likelihoods: 100%|██████████| 10/10 [00:00<00:00, 209.05it/s]


Stationary GP NLL: 4.543677857164826
PixelCNN NLL: 9.433557636439122


In [ ]:
pixel_cnn.compute_negative_log_likelihood(train_patches)